# Enable GPU

In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# Actor and Critic Network



In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

class Actor_Net(nn.Module):
  def __init__(self, input_dims, output_dims, num_neurons = 128):
    super(Actor_Net, self).__init__()
    self.fc1 = nn.Linear(input_dims, num_neurons)
    self.actor = nn.Linear(num_neurons, output_dims)
    self.log_probs = []
    self.entropies = []

  def forward(self, state):
    x = F.relu(self.fc1(state))
    x = F.softmax(self.actor(x), dim = 1)

    return x

  def get_action(self, state):
    with torch.no_grad():
      probs = self.forward(state)
      dist = Categorical(probs = probs)
      action = dist.sample()
    return action
  
  def eval_action(self, state):
    probs = self.forward(state)
    dist = Categorical(probs = probs)
    action = dist.sample().to(device)
    log_prob = dist.log_prob(action)
    entropy = dist.entropy()
    self.log_probs.append(log_prob)
    self.entropies.append(entropy)

    return action

class Critic_Net(nn.Module):
  def __init__ (self, input_dims, output_dims, num_neurons = 128):
    super(Critic_Net, self).__init__()

    self.values = []
    self.next_values = []
    
    self.fc1 = nn.Linear(input_dims, num_neurons)
    self.critic = nn.Linear(num_neurons, 1)

  def forward (self, state):
    x = F.relu(self.fc1(state))
    x = self.critic(x)

    return x

In [4]:
import torch.optim as optim
import numpy as np
import gym

class Actor_Critic_Agent(nn.Module):
  def __init__(self, input_dims, output_dims, optimizer = 'RMSprop', num_neurons = 128 , gamma = 0.99, actor_lr=0.001, critic_lr = 0.01):
    super(Actor_Critic_Agent, self).__init__()
    self.actor_net = Actor_Net(input_dims= input_dims, output_dims= output_dims, num_neurons= num_neurons).to(device)
    self.critic_net = Critic_Net(input_dims=input_dims, output_dims= output_dims, num_neurons= num_neurons).to(device)
    self.gamma = gamma
    if optimizer == 'RMSprop':
      self.actor_optimizer = optim.RMSprop(params = self.actor_net.parameters(), lr =actor_lr)
      self.critic_optimizer = optim.RMSprop(params = self.critic_net.parameters(), lr = critic_lr)
    else:
      self.actor_optimizer = optim.Adam(params = self.actor_net.parameters(), lr = actor_lr)
      self.critic_optimizer = optim.Adam(params = self.critic_net.parameters(), lr = critic_lr)

  def learn_mean(self, rewards, dones):
    value_criteration = nn.MSELoss()
    value_losses = []
    actor_losses = []
    self.critic_net.next_values = torch.cat(self.critic_net.next_values, dim = 0).squeeze(0)
    self.critic_net.values = torch.cat(self.critic_net.values, dim = 0).squeeze(0)
    self.actor_net.log_probs = torch.cat(self.actor_net.log_probs, dim = 0)
    self.actor_net.entropies = torch.cat(self.actor_net.entropies, dim = 0)

    for reward, entropy, log_prob, v, v_next, done in zip(rewards ,self.actor_net.entropies, self.actor_net.log_probs, self.critic_net.values, self.critic_net.next_values, dones):
      td_target = reward + self.gamma * v_next * done
      td_error = td_target - v
      value_loss = value_criteration(v, td_target.detach())- 0.001 * entropy.detach()
      actor_loss = - log_prob * td_error.detach() 
      value_losses.append(value_loss)
      actor_losses.append(actor_loss)

    self.critic_optimizer.zero_grad()
    value_losses = torch.stack(value_losses).sum()
    value_losses.backward()
    self.critic_optimizer.step()  

    self.actor_optimizer.zero_grad()
    actor_losses = torch.stack(actor_losses).sum()
    actor_losses.backward()
    self.actor_optimizer.step()

      
    # clear out memory 
    self.actor_net.log_probs = []
    self.actor_net.entropies = []
    self.critic_net.values = []
    self.critic_net.next_values = []



# Without Wandb

In [5]:
import gym
import time
import pdb

env = gym.make('CartPole-v1')
env.seed(543)
torch.manual_seed(543)
state_dims = env.observation_space.shape[0]
action_dims = env.action_space.n
agent = Actor_Critic_Agent(input_dims= state_dims, output_dims = action_dims)

def train():

  num_ep = 2000
  print_every = 100
  running_score = 10
  start = time.time()

  rewards = []
  dones = []

  for ep in range(1, num_ep + 1):
    state = env.reset()
    score = 0
    done = False
    rewards = []
    dones = []

    while not done:
      state = torch.tensor([state]).float().to(device)
      action = agent.actor_net.eval_action(state)
      v = agent.critic_net(state)

      next_state, reward, done, _ = env.step(action.item())
      v_next = agent.critic_net(torch.tensor([next_state]).float().to(device))
  
      agent.critic_net.values.append(v.squeeze(0))
      agent.critic_net.next_values.append(v_next.squeeze(0))
      rewards.append(reward)
      dones.append(1 - done)
      
      # update episode
      score += reward
      state = next_state

      if done:
        break

    # update agent
    #pdb.set_trace()
    agent.learn_mean(rewards,dones)
   
    # calculating score and running score
    running_score = 0.05 * score + (1 - 0.05) * running_score

    if ep % print_every == 0:
      print('episode: {}, running score: {}, time elapsed: {}'.format(ep, running_score, time.time() - start))





In [ ]:
train() #RMS

episode: 100, running score: 43.32507441570408, time elapsed: 4.842878341674805
episode: 200, running score: 129.30332722904944, time elapsed: 19.552313089370728


# Wtih wandb

In [ ]:
!pip install wandb
!wandb login


In [ ]:
import wandb
sweep_config = dict()
sweep_config['method'] = 'grid'
sweep_config['metric'] = {'name': 'running_score', 'goal': 'maximize'}
sweep_config['parameters'] = {'learning': {'value': 'learn_mean'}, 'actor_learning_rate': {'values' : [0.01, 0.001, 0.0001,0.0003,0.00001]}, 'critic_learning_rate' : {'values': [0.01, 0.001, 0.0001, 0.0003, 0.00001]}
                              , 'num_neurons': {'value': 128 }, 'optimizer': {'values' : ['RMSprop', 'Adam']}}

sweep_id = wandb.sweep(sweep_config, project = 'Advantage_Actor_Critic')

Create sweep with ID: t9gia22t
Sweep URL: https://wandb.ai/ko120/Advantage_Actor_Critic/sweeps/t9gia22t


In [ ]:
import gym 
import torch
import time
import wandb



def train():
  wandb.init(config = {'env':'CartPole-v1','algorithm:': 'Actor_Critic','architecture': 'seperate','num_laeyrs':'2'}, project = 'Advantage_Actor_Critic',group = 'Cart_128_neurons_2_layer')
  config = wandb.config

  env = gym.make('CartPole-v1')
  env.seed(543)
  torch.manual_seed(543)

  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n

  device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
  agent = Actor_Critic_Agent(input_dims= state_dim, output_dims= action_dim, optimizer = config.optimizer, num_neurons= config.num_neurons, actor_lr = config.actor_learning_rate, critic_lr = config.critic_learning_rate)


  num_ep = 3000
  print_interval = 100
  save_interval = 1000
  running_score = 10
  start = time.time()

  
  wandb.watch(agent)
  for ep in range(1,num_ep+1):
    state = env.reset()
    score = 0
    done = False
    rewards = []
    dones = []

    while not done:
      state = torch.tensor([state]).float().to(device)
      action = agent.actor_net.eval_action(state)
      v = agent.critic_net(state)

      next_state, reward, done, _ = env.step(action.item())
      v_next = agent.critic_net(torch.tensor([next_state]).float().to(device))
  
      agent.critic_net.values.append(v.squeeze(0))
      agent.critic_net.next_values.append(v_next.squeeze(0))
      rewards.append(reward)
      dones.append(1 - done)
      
      # update episode
      score += reward
      state = next_state

      if done:
        break

    # update agent
    agent.learn_mean(rewards,dones)
   
    # calculating score and running score
    running_score = 0.05 * score + (1 - 0.05) * running_score

    wandb.log({'episode': ep, 'running_score': running_score})   

    if ep % print_interval == 0:
      print('episode {} average reward {}, ended at {:.01f}'.format(ep, running_score, time.time() - start))    
    
    if ep % save_interval == 0:
      save_name_actor = 'actor_' + str(ep) + '.pt'
      torch.save(agent.actor_net.state_dict(),save_name_actor)
      save_name_critic = 'critic_' + str(ep) + '.pt'
      torch.save(agent.critic_net.state_dict(),save_name_critic)
      wandb.save(save_name_actor)
      wandb.save(save_name_critic)

    if ep == num_ep:
      dummy_input = torch.rand(1,4).to(device)
      torch.onnx.export(agent.actor_net,dummy_input,'final_model_actor.onnx')
      wandb.save('final_model_actor.onnx')
      torch.onnx.export(agent.critic_net, dummy_input, 'final_model_critic.onnx')
      wandb.save('final_model_critic.onnx')
    

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: wivnmds7 with config:
wandb: 	actor_learning_rate: 0.01
wandb: 	critic_learning_rate: 0.01
wandb: 	learning: learn_mean
wandb: 	num_neurons: 128
wandb: 	optimizer: RMSprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


episode 100 average reward 9.320976246076176, ended at 2.0
episode 200 average reward 9.500732875399857, ended at 4.2
episode 300 average reward 9.300742705361715, ended at 6.4
episode 400 average reward 9.277204480629702, ended at 8.6
episode 500 average reward 9.273707051135274, ended at 10.8
episode 600 average reward 9.327889863203684, ended at 13.0
episode 700 average reward 9.333155867973675, ended at 15.4
episode 800 average reward 9.523549421141825, ended at 17.6
episode 900 average reward 9.673708625666013, ended at 19.7
episode 1000 average reward 9.748745763117851, ended at 21.9
episode 1100 average reward 9.323193182969527, ended at 24.2
episode 1200 average reward 56.42889769119886, ended at 31.3
episode 1300 average reward 43.74196023208914, ended at 41.4
episode 1400 average reward 67.21059343899846, ended at 56.4
episode 1500 average reward 75.91453703193939, ended at 70.0
episode 1600 average reward 312.7339309544486, ended at 112.1
episode 1700 average reward 491.6694

episode,3000
running_score,497.6113
_runtime,1497
_timestamp,1627527224
_step,2999


episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
running_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▅▇█████████████████
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇██
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Agent Starting Run: 23g5a6tq with config:
wandb: 	actor_learning_rate: 0.01
wandb: 	critic_learning_rate: 0.01
wandb: 	learning: learn_mean
wandb: 	num_neurons: 128
wandb: 	optimizer: Adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: w48wnakr with config:
wandb: 	actor_learning_rate: 1e-05
wandb: 	critic_learning_rate: 1e-05
wandb: 	learning: learn_mean
wandb: 	num_neurons: 128
wandb: 	optimizer: Adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


episode 100 average reward 22.02826083168772, ended at 4.4
episode 200 average reward 23.915060653591283, ended at 8.9
episode 300 average reward 19.529958212613973, ended at 13.4
episode 400 average reward 20.012544668608147, ended at 17.8
episode 500 average reward 22.766900354863303, ended at 22.7
episode 600 average reward 23.949102055515915, ended at 27.4
episode 700 average reward 23.98487926746266, ended at 32.2
episode 800 average reward 22.257725139399007, ended at 36.6
episode 900 average reward 21.183578631694704, ended at 40.6
episode 1000 average reward 25.615198667384625, ended at 45.1
episode 1100 average reward 21.52429362781972, ended at 49.3
episode 1200 average reward 20.90847775076167, ended at 53.8
episode 1300 average reward 21.881818278107996, ended at 58.2
episode 1400 average reward 19.620668831303114, ended at 62.4
episode 1500 average reward 19.61684378930741, ended at 66.6
episode 1600 average reward 20.6829890430845, ended at 70.7
episode 1700 average rewar

episode,3000
running_score,21.83107
_runtime,135
_timestamp,1627547444
_step,2999


episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
running_score,▃▆▃▃▇▄▆▅▆▆▃▃▅█▄▅▅▄▂▅▂▄▅▂▅▃▇▅▃▃▃▄▄▁▆▃▃▄▂▆
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# You can see the result here!
[Report Link](https://wandb.ai/ko120/Advantage_Actor_Critic/reports/TD-Actor-Critic-Learning-rate-tune---Vmlldzo4OTIwODg)